In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-04-05 11:26:19--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   155MB/s    in 0.2s    

2024-04-05 11:26:19 (155 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Data cleaning and preprocessing
# Merge books and ratings dataframes
merged_df = pd.merge(df_books, df_ratings, on='isbn')

# Calculate rating counts for users and books
user_rating_count = merged_df['user'].value_counts()
book_rating_count = merged_df['title'].value_counts()

# Filter out users with less than 200 ratings
active_users = merged_df[merged_df['user'].isin(user_rating_count[user_rating_count >= 200].index)]

# Filter out books with less than 100 ratings
popular_books = active_users[active_users['title'].isin(book_rating_count[book_rating_count >= 100].index)]

# Calculate average rating for each book
average_rating = popular_books.groupby('title')['rating'].mean().reset_index()
average_rating.columns = ['title', 'average_rating']  # Renaming the column to avoid conflicts

# Merge average rating with original dataframe
popular_books = pd.merge(popular_books, average_rating, on='title')

# Create pivot table for user-item matrix
pivot_df = popular_books.pivot_table(index='title', columns='user', values='rating').fillna(0)

# Create Nearest Neighbors model
from sklearn.neighbors import NearestNeighbors
knn_model = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=8)
knn_model.fit(pivot_df.values)

# Function to get book recommendations
def get_recommends(book_title):
    book_index = pivot_df.index.get_loc(book_title)
    distances, indices = knn_model.kneighbors(pivot_df.iloc[book_index, :].values.reshape(1, -1), n_neighbors=6)
    recommended_books = []
    for i in range(1, len(distances.flatten())):
        recommended_books.append([pivot_df.index[indices.flatten()[i]], distances.flatten()[i]])
    return [book_title, recommended_books]

# Test the function
get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")


["Where the Heart Is (Oprah's Book Club (Paperback))",
 [['Blue Diary', 0.7145698],
  ['The Lovely Bones: A Novel', 0.7158567],
  ['I Know This Much Is True', 0.7496688],
  ['Icy Sparks', 0.7646523],
  ['The Weight of Water', 0.7678344]]]